### Imports

In [67]:
import os
import numpy as np
import pandas as pd

### Inputs

In [68]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_class = fusion_output_df['class'].values
fusion_output_confidence = fusion_output_df['confidence'].values

In [69]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

In [70]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_apmr_edge = radnet_input_df['apmr_edge'].values
radnet_input_mfsr_edge = radnet_input_df['mfsr_edge'].values
radnet_input_idg_global = radnet_input_df['idg_global'].values

### Kalman Filter

In [71]:
def kalman_filter(z, Q=1e-4, R=1e-2):
    n = len(z)
    xhat = np.zeros(n)
    P = np.zeros(n)
    xhat[0] = z[0]
    P[0] = 1.0
    for k in range(1, n):
        # previsão
        xhatminus = xhat[k-1]
        Pminus = P[k-1] + Q
        # atualização
        K = Pminus / (Pminus + R)
        xhat[k] = xhatminus + K * (z[k] - xhatminus)
        P[k] = (1 - K) * Pminus
    return xhat

In [72]:
confidence_fusion = np.where(fusion_output_class == 1,
    fusion_output_confidence,
    1.0 - fusion_output_confidence
)

In [73]:
kalman = kalman_filter(confidence_fusion)

# classe final a partir do filtrado
cls = (kalman >= 0.5).astype(int)

# confiança orientada à classe escolhida
conf = np.where(cls == 1, kalman, 1.0 - kalman)

In [74]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'APMR':<6} {'MFSR':<6} {'IDG':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*75}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_apmr_edge[i]:<6.2f} {radnet_input_mfsr_edge[i]:<6.2f} {radnet_input_idg_global[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   APMR   MFSR   IDG    Classe   Status   Confiança 
---------------------------------------------------------------------------
-72    8.6   83     0.69   0.82   0.91   0.89   1        Bom      0.52      
-84    3.6   37     0.71   0.87   0.85   0.84   1        Bom      0.51      
-89    8.8   63     0.45   0.50   0.77   0.57   0        Ruim     0.55      
-36    3.0   22     0.20   0.90   0.82   0.79   1        Bom      0.60      
-58    4.8   53     0.75   0.82   0.90   0.76   1        Bom      0.58      
-57    1.5   22     0.37   0.79   0.78   0.75   1        Bom      0.64      
-83    7.4   89     0.37   0.82   0.74   0.67   1        Bom      0.61      
-67    6.5   86     0.51   0.91   0.95   0.68   1        Bom      0.59      
-50    3.6   23     0.26   0.81   0.94   0.82   1        Bom      0.64      
-68    5.2   40     0.90   0.54   0.54   0.46   1        Bom      0.60      
-87    9.6   65     0.66   0.57   0.62   0.46   1        Bom      0.56      


### Save

In [75]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
